# Multimodal Analytics with Cortex AISQL

This notebook demonstrates:
- **AI_COMPLETE** with images
- **AI_TRANSCRIBE** for audio files
- **AI_FILTER** with images for content-based filtering
- **AI_PARSE_DOCUMENT** for OCR
- **TO_FILE** for file references


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from snowflake.snowpark import Session
from IPython.display import display, Markdown, HTML

# Try to get active session (for Snowflake Notebooks)
# Otherwise, connect using ~/.snowflake/connections.toml
try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    print("✅ Connected using Snowflake Notebooks session")
except:
    import toml
    from pathlib import Path
    toml_path = Path.home() / ".snowflake" / "connections.toml"
    if toml_path.exists():
        connections = toml.load(toml_path)
        connection_name = "default"
        if connection_name in connections:
            conn_params = connections[connection_name]
            session = Session.builder.configs(conn_params).create()
            print(f"✅ Connected using connection profile: {connection_name}")
        else:
            raise Exception(f"❌ Connection '{connection_name}' not found")
    else:
        raise Exception(f"❌ Connection file not found at {toml_path}")

session.sql("USE DATABASE AISQL_DB").collect()
session.sql("USE SCHEMA AISQL_SCHEMA").collect()
session.sql("USE WAREHOUSE AISQL_WH").collect()


✅ Connected using connection profile: vinodshiv


[Row(status='Statement executed successfully.')]

## 1. AI_COMPLETE with Images

Analyze screenshots and images using vision models


In [2]:
# Analyze images with AI_COMPLETE
sql = """
SELECT 
    relative_path,
    AI_COMPLETE('pixtral-large', 
        'Describe this image and identify any issues or errors visible.', 
        img_file) as image_analysis
FROM images
WHERE relative_path LIKE 'screenshot%'
LIMIT 5
"""

df_images = session.sql(sql).to_pandas()
display(Markdown("### Image Analysis Results"))
display(df_images)


### Image Analysis Results

,RELATIVE_PATH,IMAGE_ANALYSIS
0,screenshot1.png,The image shows a checkout page for purchasin...
1,screenshot2.png,The image shows a webpage interface for a che...
2,screenshot3.png,The image shows a checkout page for purchasin...
3,screenshot4.png,The image shows a login interface with a few ...
4,screenshot5.png,The image shows a smartphone screen displayin...


## 2. AI_TRANSCRIBE for Audio

Transcribe voicemail audio files


In [3]:
# Transcribe audio files
sql = """
SELECT 
    relative_path,
    AI_TRANSCRIBE(audio_file)['text'] as transcription,
    AI_SENTIMENT(AI_TRANSCRIBE(audio_file)['text'])['categories'][0]['sentiment']::STRING as overall_sentiment
FROM voicemails
LIMIT 10
"""

df_audio = session.sql(sql).to_pandas()
display(Markdown("### Audio Transcriptions"))
display(df_audio)

# Sentiment distribution
if len(df_audio) > 0:
    sentiment_counts = df_audio['OVERALL_SENTIMENT'].value_counts()
    display(Markdown("**Voicemail Sentiment Summary:**"))
    for sentiment, count in sentiment_counts.items():
        display(Markdown(f"- {sentiment.capitalize()}: {count}"))


### Audio Transcriptions

,RELATIVE_PATH,TRANSCRIPTION,OVERALL_SENTIMENT
0,c10128_neutral_uk.mp3,"""Your festival wristband activation system wor...",neutral
1,c10296_neutral_uk.mp3,"""Your festival food and beverage voucher syste...",mixed
2,c10351_negative_au.mp3,"""The mobile ticketing app performed poorly at ...",negative
3,c10483_positive_uk.mp3,"""The family package for the holiday concert ex...",positive
4,c10546_positive_uk.mp3,"""The membership program for your concert serie...",positive
5,c10625_negative_uk.mp3,"""I experienced significant issues with your ti...",negative
6,c10743_neutral_uk.mp3,"""The group discount for the jazz concert serie...",mixed
7,c10838_negative_au.mp3,"""I'm writing to express my dissatisfaction wit...",negative
8,c10941_negative_uk.mp3,"""The premium lounge access sold with platinum ...",negative
9,c10982_positive_uk.mp3,"""The gifting option on your ticketing platform...",positive


**Voicemail Sentiment Summary:**

- Negative: 4

- Positive: 3

- Mixed: 2

- Neutral: 1

## 3. AI_FILTER with Images

Filter and classify images based on their content using natural language predicates


In [4]:
# Filter images based on content - Example 1: Screenshots with errors
sql = """
SELECT 
    relative_path,
    AI_FILTER('Does this image show an error message or warning?', img_file) as has_error
FROM images
WHERE AI_FILTER('Does this image show an error message or warning?', img_file) = TRUE
LIMIT 10
"""

df_errors = session.sql(sql).to_pandas()
display(Markdown("### Images with Errors or Warnings"))
display(df_errors)

# Filter images - Example 2: Using PROMPT for more complex filtering
sql = """
SELECT 
    relative_path,
    AI_FILTER(PROMPT('{0} contains a user interface element', img_file)) as has_ui
FROM images
WHERE AI_FILTER(PROMPT('{0} contains a user interface element', img_file)) = TRUE
LIMIT 10
"""

df_ui = session.sql(sql).to_pandas()
display(Markdown("### Images with UI Elements"))
display(df_ui)

# Count images by category
sql = """
SELECT 
    AI_FILTER('Does this image show text or documents?', img_file) as has_text,
    COUNT(*) as count
FROM images
GROUP BY has_text
"""

df_counts = session.sql(sql).to_pandas()
display(Markdown("### Image Content Distribution"))
display(df_counts)

# Visualize
fig = px.pie(df_counts, 
             values='COUNT', 
             names='HAS_TEXT',
             title='Images with Text vs Without',
             color='HAS_TEXT',
             color_discrete_map={True: 'blue', False: 'gray'})
fig.update_layout(height=400)
fig.show()


### Images with Errors or Warnings

,RELATIVE_PATH,HAS_ERROR
0,01a_data_engineer_before.jpg,True
1,03a_ml_engineer_before.jpg,True
2,04a_data_analyst_before.jpg,True
3,screenshot1.png,True
4,screenshot2.png,True
5,screenshot3.png,True
6,screenshot4.png,True
7,screenshot5.png,True
8,screenshot6.png,True
9,screenshot8.png,True


### Images with UI Elements

,RELATIVE_PATH,HAS_UI
0,01b_data_engineer_after.png,True
1,01c_data_engineer_before_after_metrics.jpg,True
2,02b_data_scientist_after.jpg,True
3,02c_data_scientist_before_after_metrics.jpg,True
4,04b_data_analyst_after.jpg,True
5,04c_data_analyst_before_after_metrics.jpg,True
6,05a_data_ops_before.jpg,True
7,05b_data_ops_after.jpg,True
8,screenshot1.png,True
9,screenshot2.png,True


### Image Content Distribution

,HAS_TEXT,COUNT
0,True,25
1,False,1


## 4. AI_PARSE_DOCUMENT for OCR

Extract text from images using OCR


In [5]:
# Parse documents with OCR
# AI_PARSE_DOCUMENT returns a VARCHAR (JSON string)
# Access the JSON fields directly without PARSE_JSON
sql = """
SELECT 
    relative_path,
    AI_PARSE_DOCUMENT(img_file, OBJECT_CONSTRUCT('mode', 'OCR'))['content']::STRING as extracted_text,
    LENGTH(AI_PARSE_DOCUMENT(img_file, OBJECT_CONSTRUCT('mode', 'OCR'))['content']::STRING) as text_length
FROM images
WHERE relative_path LIKE 'screenshot%'
LIMIT 5
"""

df_ocr = session.sql(sql).to_pandas()
display(Markdown("### OCR Extracted Text"))

for idx, row in df_ocr.iterrows():
    display(Markdown(f"#### {row['RELATIVE_PATH']} ({row['TEXT_LENGTH']} chars)"))
    display(Markdown(f"```\n{row['EXTRACTED_TEXT']}\n```"))
    display(Markdown("---"))


### OCR Extracted Text

#### screenshot1.png (271 chars)

```
Concerts Sports Arts & Theater Help Sign In
Checkout
Transaction declined by bank. Please try another payment method.
Foo Fighters Payment
City Arena Card Number Sat, June 15 -8:00 PM
Expiration Date CVV
1 Ticket $120,00
Service Fee $15.00 Place Order
Order Total $135.00
```

---

#### screenshot2.png (89 chars)

...

```
Login
Account Locked
Email address
Password
Log in
Forgot password?
```

---

#### screenshot5.png (57 chars)

```
6:40
Ticket Booking
ERROR
An unexpected error occurred
OK
```

---